# Fases 3-5: Extracción de Características TDA y Clasificación (VERSIÓN MEJORADA)
## Proyecto 1.b - Análisis de Grafos de Conectividad

**Objetivo del Proyecto**:
Determinar si el audio lento induce patrones de conectividad EEG diferentes comparado con el audio rápido en infantes, utilizando Análisis de Datos Topológicos (TDA) en grafos de conectividad funcional.

**Hipótesis**:
Audio lento → patrones de conectividad diferentes → firmas topológicas distintas → clasificable

**Pipeline**:
- **Fase 3**: Extraer características topológicas de matrices de distancia usando Ripser
- **Fase 4**: Entrenar clasificadores con validación cruzada a nivel de sujeto
- **Fase 5**: Validación estadística y prueba de hipótesis

**Requisito Clave**: Separación por sujetos para evitar fuga de datos (múltiples grabaciones por sujeto)

**Cambios respecto a v1**:
- Orden de secciones corregido
- Matriz de confusión basada en CV (no conjunto de entrenamiento)
- Visualización de diagramas de persistencia
- Análisis de importancia de características por banda
- Mejor manejo de casos borde
- Validación de datos añadida
- Mejor documentación e interpretación

## 1. Configuración e Importaciones

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
import warnings
from scipy import stats
from collections import defaultdict

warnings.filterwarnings("ignore")

# Librerías TDA
from ripser import ripser
import persim

# Aprendizaje Automático
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    cross_val_score,
    cross_val_predict,
    LeaveOneGroupOut,
    GroupKFold,
)
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    f1_score,
)
from sklearn.preprocessing import StandardScaler

# Configurar estilo
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")
np.random.seed(42)

print("=" * 60)
print("Pipeline de Clasificación TDA-EEG v2.0")
print("=" * 60)
print("Librerías importadas exitosamente")

## 2. Configuración y Rutas de Datos

In [ ]:
# Rutas
DIRECTORIO_GRAFOS = Path("graphs")
DIRECTORIO_CARACTERISTICAS = Path("caracteristicas")
DIRECTORIO_RESULTADOS = Path("resultados")
DIRECTORIO_CARACTERISTICAS.mkdir(exist_ok=True)
DIRECTORIO_RESULTADOS.mkdir(exist_ok=True)

# Bandas de frecuencia - cada una captura diferente actividad neural
BANDAS_FRECUENCIA = ["delta", "theta", "alpha", "beta", "gamma"]
RANGOS_BANDAS = {
    "delta": "0.5-4 Hz (sueño profundo, atención)",
    "theta": "4-8 Hz (somnolencia, memoria)",
    "alpha": "8-13 Hz (relajación, inhibición)",
    "beta": "13-30 Hz (alerta, pensamiento activo)",
    "gamma": "30-100 Hz (percepción, cognición)",
}

# Parámetros TDA
DIMENSION_MAXIMA = 1  # Calcular H0 (componentes conexas) y H1 (ciclos/agujeros)
# Nota: LONGITUD_ARISTA_MAXIMA debe ajustarse según tu métrica de distancia
# Si usas distancia de correlación (1 - |corr|): rango es [0, 1], usar ~1.0
# Si usas otras métricas, ajustar correspondientemente
LONGITUD_ARISTA_MAXIMA = 2.0  # Valor máximo de filtración para complejo de Rips

# Parámetros de clasificación
N_PARTICIONES = 5  # Número de folds para GroupKFold CV
N_PERMUTACIONES = 1000  # Para prueba de permutación
N_BOOTSTRAP = 1000  # Para intervalos de confianza
SEMILLA_ALEATORIA = 42

print("Configuración:")
print(f"  Directorio de grafos: {DIRECTORIO_GRAFOS}")
print(f"  Salida de características: {DIRECTORIO_CARACTERISTICAS}")
print(f"  Salida de resultados: {DIRECTORIO_RESULTADOS}")
print(f"\nBandas de frecuencia:")
for banda, descripcion in RANGOS_BANDAS.items():
    print(f"  {banda}: {descripcion}")
print(f"\nParámetros TDA:")
print(f"  Dimensión homológica máxima: H{DIMENSION_MAXIMA}")
print(f"  Longitud máxima de arista (filtración): {LONGITUD_ARISTA_MAXIMA}")
print(f"\nParámetros de clasificación:")
print(f"  Particiones CV: {N_PARTICIONES}")
print(f"  Iteraciones de permutación: {N_PERMUTACIONES}")
print(f"  Iteraciones bootstrap: {N_BOOTSTRAP}")

# Fase 3: Análisis de Datos Topológicos

## 3.1 Entendiendo el Enfoque TDA

**¿Por qué TDA para conectividad EEG?**
- Los grafos de conectividad EEG capturan relaciones funcionales entre regiones cerebrales
- Las métricas tradicionales de grafos (densidad, clustering) pierden información estructural
- TDA captura características topológicas multi-escala que persisten a través de valores de filtración

**Qué calculamos:**
- **H0 (componentes conexas)**: Cómo el grafo se fragmenta/conecta en diferentes umbrales
- **H1 (ciclos/agujeros)**: Patrones circulares en conectividad que podrían indicar bucles de retroalimentación

**Conexión con la hipótesis:**
- El audio lento podría inducir actividad cerebral más sincronizada → topología de conectividad diferente
- Estas diferencias deberían manifestarse como diagramas de persistencia diferentes

## 3.2 Cálculo del Diagrama de Persistencia

In [ ]:
def validar_matriz_distancia(matriz_distancia, nombre=""):
    """
    Validar que una matriz sea una matriz de distancia válida.
    
    Requisitos:
    - Matriz cuadrada
    - Simétrica (o casi simétrica)
    - Valores no negativos
    - Diagonal cero
    
    Parámetros:
    -----------
    matriz_distancia : ndarray
        Matriz a validar
    nombre : str
        Nombre para mensajes de error
        
    Retorna:
    --------
    es_valida : bool
    problemas : lista de str
    """
    problemas = []
    
    # Verificar cuadrada
    if matriz_distancia.ndim != 2:
        problemas.append(f"No es 2D: forma={matriz_distancia.shape}")
        return False, problemas
    
    n, m = matriz_distancia.shape
    if n != m:
        problemas.append(f"No es cuadrada: forma=({n}, {m})")
        return False, problemas
    
    # Verificar simetría (con tolerancia para punto flotante)
    if not np.allclose(matriz_distancia, matriz_distancia.T, rtol=1e-5, atol=1e-8):
        diferencia_max = np.max(np.abs(matriz_distancia - matriz_distancia.T))
        problemas.append(f"No es simétrica: asimetría máxima={diferencia_max:.6f}")
    
    # Verificar no negativo
    if np.any(matriz_distancia < -1e-10):  # Pequeña tolerancia para errores numéricos
        valor_min = np.min(matriz_distancia)
        problemas.append(f"Valores negativos presentes: mín={valor_min:.6f}")
    
    # Verificar diagonal
    diagonal = np.diag(matriz_distancia)
    if not np.allclose(diagonal, 0, atol=1e-10):
        diagonal_max = np.max(np.abs(diagonal))
        problemas.append(f"Diagonal no cero: máx={diagonal_max:.6f}")
    
    # Verificar NaN/Inf
    if np.any(np.isnan(matriz_distancia)):
        problemas.append("Contiene valores NaN")
    if np.any(np.isinf(matriz_distancia)):
        problemas.append("Contiene valores Inf")
    
    es_valida = len(problemas) == 0
    return es_valida, problemas


def calcular_diagrama_persistencia(matriz_distancia, dim_max=1, longitud_arista_max=2.0):
    """
    Calcular diagrama de persistencia desde matriz de distancia usando Ripser.
    
    El diagrama de persistencia captura características topológicas (componentes, ciclos)
    que aparecen y desaparecen al aumentar el umbral de distancia.
    
    Parámetros:
    -----------
    matriz_distancia : ndarray, forma (n, n)
        Matriz de distancia simétrica con diagonal cero
    dim_max : int
        Dimensión homológica máxima (1 = H0 y H1)
    longitud_arista_max : float
        Longitud máxima de arista para filtración del complejo de Rips
        
    Retorna:
    --------
    diagramas : lista de ndarray
        Diagramas de persistencia para cada dimensión [H0, H1, ...]
        Cada diagrama tiene forma (n_caracteristicas, 2) con pares (nacimiento, muerte)
    """
    # Asegurar que la matriz sea simétrica (promediar si hay pequeñas diferencias)
    matriz_distancia = (matriz_distancia + matriz_distancia.T) / 2
    np.fill_diagonal(matriz_distancia, 0)
    
    # Recortar valores negativos pequeños de errores numéricos
    matriz_distancia = np.maximum(matriz_distancia, 0)
    
    resultado = ripser(
        matriz_distancia,
        maxdim=dim_max,
        thresh=longitud_arista_max,
        distance_matrix=True
    )
    return resultado["dgms"]


def extraer_caracteristicas_persistencia(diagrama, nombre_dim=""):
    """
    Extraer características escalares de un diagrama de persistencia.
    
    Las características capturan diferentes aspectos de la estructura topológica:
    - Conteo: Cuántas características existen
    - Tiempos de nacimiento/muerte: Cuándo aparecen/desaparecen las características
    - Persistencia: Cuánto duran las características (muerte - nacimiento)
    - Entropía: Distribución de valores de persistencia
    
    Parámetros:
    -----------
    diagrama : ndarray, forma (n_caracteristicas, 2)
        Diagrama de persistencia con pares (nacimiento, muerte)
    nombre_dim : str
        Nombre de la dimensión para nombrar características
        
    Retorna:
    --------
    caracteristicas : dict
        Diccionario de características escalares extraídas
    """
    # Remover tiempos de muerte infinitos (características esenciales que nunca mueren)
    mascara_finita = np.isfinite(diagrama).all(axis=1)
    diagrama_finito = diagrama[mascara_finita]
    
    # Contar características esenciales (aquellas con persistencia infinita)
    n_esenciales = np.sum(~mascara_finita)
    
    if len(diagrama_finito) == 0:
        # Sin características finitas - retornar ceros
        return {
            "n_caracteristicas": 0,
            "n_esenciales": n_esenciales,
            "media_nacimiento": 0,
            "std_nacimiento": 0,
            "media_muerte": 0,
            "std_muerte": 0,
            "media_persistencia": 0,
            "std_persistencia": 0,
            "max_persistencia": 0,
            "total_persistencia": 0,
            "entropia_persistencia": 0,
        }
    
    nacimientos = diagrama_finito[:, 0]
    muertes = diagrama_finito[:, 1]
    persistencia = muertes - nacimientos
    
    # Calcular entropía de persistencia (normalizada)
    # Mayor entropía = distribución más uniforme de valores de persistencia
    if len(persistencia) > 1 and np.sum(persistencia) > 0:
        p_normalizada = persistencia / np.sum(persistencia)
        p_normalizada = p_normalizada[p_normalizada > 0]  # Remover ceros para log
        entropia = -np.sum(p_normalizada * np.log(p_normalizada + 1e-10))
        # Normalizar por entropía máxima posible
        entropia = entropia / np.log(len(persistencia) + 1e-10)
    else:
        entropia = 0
    
    caracteristicas = {
        "n_caracteristicas": len(diagrama_finito),
        "n_esenciales": n_esenciales,
        "media_nacimiento": np.mean(nacimientos),
        "std_nacimiento": np.std(nacimientos) if len(nacimientos) > 1 else 0,
        "media_muerte": np.mean(muertes),
        "std_muerte": np.std(muertes) if len(muertes) > 1 else 0,
        "media_persistencia": np.mean(persistencia),
        "std_persistencia": np.std(persistencia) if len(persistencia) > 1 else 0,
        "max_persistencia": np.max(persistencia),
        "total_persistencia": np.sum(persistencia),
        "entropia_persistencia": entropia,
    }
    
    return caracteristicas


# Probar con datos de ejemplo
print("\n" + "=" * 60)
print("Probando Pipeline TDA")
print("=" * 60)

print("\nGenerando matriz de distancia de ejemplo...")
np.random.seed(42)
dist_prueba = np.random.rand(47, 47)
dist_prueba = (dist_prueba + dist_prueba.T) / 2  # Hacer simétrica
np.fill_diagonal(dist_prueba, 0)

# Validar
es_valida, problemas = validar_matriz_distancia(dist_prueba, "prueba")
print(f"Validación de matriz de distancia: {'PASÓ' if es_valida else 'FALLÓ'}")
if problemas:
    for problema in problemas:
        print(f"  - {problema}")

# Calcular persistencia
diagramas_prueba = calcular_diagrama_persistencia(dist_prueba, DIMENSION_MAXIMA, LONGITUD_ARISTA_MAXIMA)
print(f"\nDiagramas de persistencia calculados:")
print(f"  H0 (componentes conexas): {len(diagramas_prueba[0])} características")
print(f"  H1 (ciclos/agujeros): {len(diagramas_prueba[1])} características")

# Extraer características
caract_h0 = extraer_caracteristicas_persistencia(diagramas_prueba[0], "H0")
caract_h1 = extraer_caracteristicas_persistencia(diagramas_prueba[1], "H1")
print(f"\nCaracterísticas extraídas:")
print(f"  H0: {len(caract_h0)} características escalares")
print(f"  H1: {len(caract_h1)} características escalares")
print(f"  Total por banda: {len(caract_h0) + len(caract_h1)} características")

## 3.3 Visualizar Diagramas de Persistencia

In [ ]:
def graficar_diagrama_persistencia(diagramas, titulo="Diagrama de Persistencia", ax=None):
    """
    Graficar diagrama de persistencia con tiempos de nacimiento vs muerte.
    
    Puntos lejos de la diagonal = características persistentes (importantes)
    Puntos cerca de la diagonal = ruido (características de corta vida)
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
    
    colores = ['blue', 'orange', 'green']
    etiquetas = ['H0 (componentes)', 'H1 (ciclos)', 'H2']
    
    valor_max = 0
    for dim, dgm in enumerate(diagramas):
        if len(dgm) > 0:
            mascara_finita = np.isfinite(dgm).all(axis=1)
            dgm_finito = dgm[mascara_finita]
            
            if len(dgm_finito) > 0:
                ax.scatter(
                    dgm_finito[:, 0], dgm_finito[:, 1],
                    c=colores[dim], label=etiquetas[dim], alpha=0.6, s=50
                )
                valor_max = max(valor_max, dgm_finito.max())
            
            # Graficar características esenciales (muerte infinita) arriba
            esenciales = dgm[~mascara_finita]
            if len(esenciales) > 0:
                ax.scatter(
                    esenciales[:, 0], 
                    [valor_max * 1.1] * len(esenciales),
                    c=colores[dim], marker='^', s=100, alpha=0.8
                )
    
    # Línea diagonal (nacimiento = muerte)
    ax.plot([0, valor_max * 1.1], [0, valor_max * 1.1], 'k--', alpha=0.3, label='Diagonal')
    
    ax.set_xlabel('Nacimiento', fontsize=12)
    ax.set_ylabel('Muerte', fontsize=12)
    ax.set_title(titulo, fontsize=14, fontweight='bold')
    ax.legend(loc='lower right')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    return ax


# Visualizar diagrama de persistencia de prueba
fig, ax = plt.subplots(figsize=(8, 8))
graficar_diagrama_persistencia(diagramas_prueba, "Diagrama de Persistencia de Ejemplo (Datos Aleatorios)", ax)
plt.tight_layout()
plt.savefig(DIRECTORIO_RESULTADOS / "diagrama_persistencia_ejemplo.png", dpi=150)
plt.show()

print("Explicación del diagrama de persistencia:")
print("  - Cada punto representa una característica topológica")
print("  - Eje X (nacimiento): umbral de distancia donde aparece la característica")
print("  - Eje Y (muerte): umbral de distancia donde desaparece la característica")
print("  - Distancia a la diagonal = persistencia = importancia")
print("  - H0: componentes conexas (cómo se fragmenta el grafo)")
print("  - H1: ciclos/agujeros (patrones de conectividad circular)")

## 4. Procesar Todos los Archivos y Extraer Características

In [ ]:
def procesar_caracteristicas_archivo(directorio_archivo, bandas_frecuencia, dim_max=1, longitud_arista_max=2.0, verbose=False):
    """
    Procesar un archivo: calcular características TDA para todas las ventanas y bandas.
    
    Para cada banda de frecuencia:
    1. Cargar matrices de distancia para todas las ventanas de tiempo
    2. Calcular diagrama de persistencia para cada ventana
    3. Extraer características escalares de cada diagrama
    4. Agregar características a través de ventanas (media/std)
    
    Parámetros:
    -----------
    directorio_archivo : Path
        Directorio que contiene matrices de distancia para un archivo
    bandas_frecuencia : lista
        Lista de nombres de bandas de frecuencia
    dim_max : int
        Dimensión homológica máxima
    longitud_arista_max : float
        Longitud máxima de arista para filtración de Rips
    verbose : bool
        Imprimir progreso detallado
        
    Retorna:
    --------
    diccionario_caracteristicas : dict
        Características agregadas para este archivo
    metadatos : dict
        Información sobre el procesamiento (n_ventanas por banda, problemas de validación)
    """
    caracteristicas_archivo = {}
    metadatos = {"n_ventanas": {}, "problemas_validacion": []}
    
    for banda in bandas_frecuencia:
        archivo_dist = directorio_archivo / f"{banda}_distances.npy"
        if not archivo_dist.exists():
            if verbose:
                print(f"  Advertencia: {banda}_distances.npy no encontrado")
            metadatos["n_ventanas"][banda] = 0
            continue
        
        # Cargar matrices de distancia (n_ventanas, n_electrodos, n_electrodos)
        try:
            matrices_distancia = np.load(archivo_dist)
        except Exception as e:
            metadatos["problemas_validacion"].append(f"{banda}: error de carga - {e}")
            continue
        
        n_ventanas = matrices_distancia.shape[0]
        metadatos["n_ventanas"][banda] = n_ventanas
        
        if n_ventanas == 0:
            if verbose:
                print(f"  Advertencia: {banda} tiene 0 ventanas")
            continue
        
        # Validar primera matriz
        es_valida, problemas = validar_matriz_distancia(matrices_distancia[0], f"{banda}[0]")
        if not es_valida:
            metadatos["problemas_validacion"].extend([f"{banda}: {p}" for p in problemas])
            # Continuar de todos modos - intentaremos arreglar en calcular_diagrama_persistencia
        
        # Recolectar características de todas las ventanas
        lista_caracteristicas_h0 = []
        lista_caracteristicas_h1 = []
        
        for i in range(n_ventanas):
            matriz_dist = matrices_distancia[i]
            
            try:
                # Calcular diagramas de persistencia
                diagramas = calcular_diagrama_persistencia(
                    matriz_dist, dim_max, longitud_arista_max
                )
                
                # Extraer características
                caract_h0 = extraer_caracteristicas_persistencia(diagramas[0], "H0")
                caract_h1 = extraer_caracteristicas_persistencia(diagramas[1], "H1")
                
                lista_caracteristicas_h0.append(caract_h0)
                lista_caracteristicas_h1.append(caract_h1)
                
            except Exception as e:
                if verbose:
                    print(f"  Error en {banda} ventana {i}: {e}")
                continue
        
        # Verificar si obtuvimos características válidas
        if len(lista_caracteristicas_h0) == 0:
            if verbose:
                print(f"  Advertencia: Sin ventanas válidas para {banda}")
            continue
        
        # Agregar a través de ventanas (media y std)
        for nombre_caract in lista_caracteristicas_h0[0].keys():
            valores_h0 = [c[nombre_caract] for c in lista_caracteristicas_h0]
            caracteristicas_archivo[f"{banda}_h0_{nombre_caract}_media"] = np.mean(valores_h0)
            caracteristicas_archivo[f"{banda}_h0_{nombre_caract}_std"] = np.std(valores_h0)
            
            valores_h1 = [c[nombre_caract] for c in lista_caracteristicas_h1]
            caracteristicas_archivo[f"{banda}_h1_{nombre_caract}_media"] = np.mean(valores_h1)
            caracteristicas_archivo[f"{banda}_h1_{nombre_caract}_std"] = np.std(valores_h1)
    
    return caracteristicas_archivo, metadatos


# Probar en un archivo
print("\n" + "=" * 60)
print("Probando Extracción de Características en Datos Reales")
print("=" * 60)

directorios_lento = list((DIRECTORIO_GRAFOS / "slow").iterdir())
if len(directorios_lento) > 0:
    directorio_grafo_prueba = directorios_lento[0]
    print(f"Probando en: {directorio_grafo_prueba.name}")
    
    caracteristicas_prueba, metadatos_prueba = procesar_caracteristicas_archivo(
        directorio_grafo_prueba, BANDAS_FRECUENCIA, DIMENSION_MAXIMA, LONGITUD_ARISTA_MAXIMA, verbose=True
    )
    
    print(f"\nResultados de extracción de características:")
    print(f"  Total de características: {len(caracteristicas_prueba)}")
    print(f"  Ventanas por banda: {metadatos_prueba['n_ventanas']}")
    if metadatos_prueba['problemas_validacion']:
        print(f"  Problemas de validación: {metadatos_prueba['problemas_validacion']}")
    print(f"  Características de ejemplo: {list(caracteristicas_prueba.keys())[:5]}")
else:
    print("No se encontraron datos en el directorio graphs/slow")

## 5. Crear Conjunto de Datos Completo

In [ ]:
def crear_conjunto_datos(dir_grafos_lento, dir_grafos_rapido, bandas_frecuencia, dim_max=1, longitud_arista_max=2.0):
    """
    Crear conjunto de datos completo desde todos los archivos.
    
    Extrae IDs de sujetos de los nombres de archivo para validación cruzada apropiada.
    Formato de nombre de archivo esperado: bbXX_utYY (sujeto_ensayo)
    
    Retorna:
    --------
    X : ndarray (n_muestras, n_caracteristicas)
    y : ndarray (n_muestras,) - 0=lento, 1=rápido
    sujetos : ndarray (n_muestras,) - IDs de sujetos
    nombres_caracteristicas : lista
    nombres_archivos : lista
    todos_metadatos : lista de dicts
    """
    todas_caracteristicas = []
    todas_etiquetas = []
    todos_sujetos = []
    todos_nombres_archivos = []
    todos_metadatos = []
    
    # Procesar archivos lentos (etiqueta = 0)
    directorios_lento = sorted([d for d in dir_grafos_lento.iterdir() if d.is_dir()])
    print(f"\nProcesando {len(directorios_lento)} archivos de audio LENTO...")
    
    for directorio_archivo in tqdm(directorios_lento, desc="Lento"):
        try:
            caracteristicas, metadatos = procesar_caracteristicas_archivo(
                directorio_archivo, bandas_frecuencia, dim_max, longitud_arista_max
            )
            if len(caracteristicas) > 0:
                todas_caracteristicas.append(caracteristicas)
                todas_etiquetas.append(0)  # lento = 0
                
                # Extraer ID de sujeto (formato: bbXX_utYY)
                nombre_archivo = directorio_archivo.name
                partes = nombre_archivo.split("_")
                id_sujeto = partes[0] if len(partes) > 0 else nombre_archivo
                
                todos_sujetos.append(id_sujeto)
                todos_nombres_archivos.append(nombre_archivo)
                todos_metadatos.append(metadatos)
        except Exception as e:
            print(f"Error procesando {directorio_archivo.name}: {e}")
    
    # Procesar archivos rápidos (etiqueta = 1)
    directorios_rapido = sorted([d for d in dir_grafos_rapido.iterdir() if d.is_dir()])
    print(f"Procesando {len(directorios_rapido)} archivos de audio RÁPIDO...")
    
    for directorio_archivo in tqdm(directorios_rapido, desc="Rápido"):
        try:
            caracteristicas, metadatos = procesar_caracteristicas_archivo(
                directorio_archivo, bandas_frecuencia, dim_max, longitud_arista_max
            )
            if len(caracteristicas) > 0:
                todas_caracteristicas.append(caracteristicas)
                todas_etiquetas.append(1)  # rápido = 1
                
                nombre_archivo = directorio_archivo.name
                partes = nombre_archivo.split("_")
                id_sujeto = partes[0] if len(partes) > 0 else nombre_archivo
                
                todos_sujetos.append(id_sujeto)
                todos_nombres_archivos.append(nombre_archivo)
                todos_metadatos.append(metadatos)
        except Exception as e:
            print(f"Error procesando {directorio_archivo.name}: {e}")
    
    # Convertir a arrays
    df_caracteristicas = pd.DataFrame(todas_caracteristicas)
    nombres_caracteristicas = list(df_caracteristicas.columns)
    X = df_caracteristicas.values
    y = np.array(todas_etiquetas)
    sujetos = np.array(todos_sujetos)
    
    print(f"\n{'=' * 60}")
    print("Resumen del Conjunto de Datos")
    print("=" * 60)
    print(f"Total de muestras: {X.shape[0]}")
    print(f"Total de características: {X.shape[1]}")
    print(f"  Características por banda: {X.shape[1] // len(bandas_frecuencia)}")
    print(f"\nDistribución de clases:")
    print(f"  Lento (0): {np.sum(y == 0)} muestras ({np.sum(y == 0) / len(y) * 100:.1f}%)")
    print(f"  Rápido (1): {np.sum(y == 1)} muestras ({np.sum(y == 1) / len(y) * 100:.1f}%)")
    print(f"\nDistribución de sujetos:")
    print(f"  Sujetos únicos: {len(np.unique(sujetos))}")
    
    return X, y, sujetos, nombres_caracteristicas, todos_nombres_archivos, todos_metadatos


# Crear conjunto de datos
print("\n" + "=" * 60)
print("Creando Conjunto de Datos Completo")
print("=" * 60)

X, y, sujetos, nombres_caracteristicas, nombres_archivos, todos_metadatos = crear_conjunto_datos(
    DIRECTORIO_GRAFOS / "slow",
    DIRECTORIO_GRAFOS / "fast",
    BANDAS_FRECUENCIA,
    DIMENSION_MAXIMA,
    LONGITUD_ARISTA_MAXIMA
)

# Guardar conjunto de datos
np.save(DIRECTORIO_CARACTERISTICAS / "X.npy", X)
np.save(DIRECTORIO_CARACTERISTICAS / "y.npy", y)
np.save(DIRECTORIO_CARACTERISTICAS / "sujetos.npy", sujetos)

with open(DIRECTORIO_CARACTERISTICAS / "nombres_caracteristicas.txt", "w") as f:
    for nombre in nombres_caracteristicas:
        f.write(f"{nombre}\n")

with open(DIRECTORIO_CARACTERISTICAS / "nombres_archivos.txt", "w") as f:
    for nombre in nombres_archivos:
        f.write(f"{nombre}\n")

print(f"\nConjunto de datos guardado en {DIRECTORIO_CARACTERISTICAS}")

## 6. Preprocesamiento y Validación de Datos

In [ ]:
print("\n" + "=" * 60)
print("Preprocesamiento de Datos")
print("=" * 60)

# Verificar valores NaN/Inf
print("\nVerificando valores inválidos...")
conteo_nan = np.isnan(X).sum()
conteo_inf = np.isinf(X).sum()
print(f"  Valores NaN: {conteo_nan}")
print(f"  Valores Inf: {conteo_inf}")

# Manejar valores problemáticos
mascara_nan = np.isnan(X).any(axis=1)
mascara_inf = np.isinf(X).any(axis=1)
mascara_valida = ~(mascara_nan | mascara_inf)

n_eliminados = (~mascara_valida).sum()
if n_eliminados > 0:
    print(f"\nEliminando {n_eliminados} muestras con valores inválidos...")
    X = X[mascara_valida]
    y = y[mascara_valida]
    sujetos = sujetos[mascara_valida]
    nombres_archivos = [f for f, v in zip(nombres_archivos, mascara_valida) if v]

print(f"\nConjunto de datos limpio: {X.shape[0]} muestras, {X.shape[1]} características")

# Analizar distribuciones de características
print("\nEstadísticas de características:")
print(f"  Valor mínimo: {X.min():.4f}")
print(f"  Valor máximo: {X.max():.4f}")
print(f"  Media: {X.mean():.4f}")
print(f"  Desviación estándar: {X.std():.4f}")

# Verificar características constantes (varianza cero)
stds_caracteristicas = X.std(axis=0)
caracteristicas_constantes = stds_caracteristicas < 1e-10
n_constantes = caracteristicas_constantes.sum()
if n_constantes > 0:
    print(f"\nAdvertencia: {n_constantes} características tienen varianza cero")
    print("  Estas no serán informativas para la clasificación")
    nombres_constantes = [nombres_caracteristicas[i] for i in np.where(caracteristicas_constantes)[0]]
    print(f"  Características constantes: {nombres_constantes[:5]}...")

# Estandarizar características
escalador = StandardScaler()
X_escalado = escalador.fit_transform(X)

print("\nCaracterísticas estandarizadas (media=0, std=1)")

## 7. Análisis de Distribución de Sujetos

In [ ]:
print("\n" + "=" * 60)
print("Análisis de Distribución de Sujetos")
print("=" * 60)

# Crear resumen a nivel de sujeto
df_sujetos = pd.DataFrame({
    "sujeto": sujetos,
    "etiqueta": y,
    "nombre_etiqueta": ["lento" if e == 0 else "rapido" for e in y]
})

# Muestras por sujeto
conteo_sujetos = df_sujetos.groupby("sujeto").size()
print(f"\nMuestras por sujeto:")
print(f"  Media: {conteo_sujetos.mean():.1f}")
print(f"  Mediana: {conteo_sujetos.median():.1f}")
print(f"  Mínimo: {conteo_sujetos.min()}")
print(f"  Máximo: {conteo_sujetos.max()}")

# Distribución de etiquetas por sujeto
etiquetas_sujeto = df_sujetos.groupby("sujeto")["etiqueta"].agg(["count", "sum", "mean"])
etiquetas_sujeto.columns = ["total", "n_rapido", "prop_rapido"]
etiquetas_sujeto["n_lento"] = etiquetas_sujeto["total"] - etiquetas_sujeto["n_rapido"]

print(f"\nDistribución de etiquetas por sujeto:")
print(etiquetas_sujeto.describe())

# Verificar si los sujetos tienen ambas condiciones (diseño intra-sujeto)
sujetos_mixtos = etiquetas_sujeto[(etiquetas_sujeto["n_lento"] > 0) & (etiquetas_sujeto["n_rapido"] > 0)]
print(f"\nSujetos con ambas condiciones: {len(sujetos_mixtos)} / {len(etiquetas_sujeto)}")

# Visualizar
fig, ejes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Muestras por sujeto
ax1 = ejes[0]
conteo_sujetos.plot(kind="bar", ax=ax1, color="steelblue", alpha=0.7)
ax1.set_xlabel("ID de Sujeto")
ax1.set_ylabel("Número de Muestras")
ax1.set_title("Muestras por Sujeto", fontweight="bold")
ax1.axhline(conteo_sujetos.mean(), color="red", linestyle="--", label=f"Media: {conteo_sujetos.mean():.1f}")
ax1.legend()
ax1.tick_params(axis='x', rotation=45)

# Gráfico 2: Distribución de etiquetas
ax2 = ejes[1]
etiquetas_sujeto[["n_lento", "n_rapido"]].plot(kind="bar", stacked=True, ax=ax2, color=["blue", "orange"], alpha=0.7)
ax2.set_xlabel("ID de Sujeto")
ax2.set_ylabel("Número de Muestras")
ax2.set_title("Distribución de Clases por Sujeto", fontweight="bold")
ax2.legend(["Lento", "Rápido"])
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(DIRECTORIO_RESULTADOS / "distribucion_sujetos.png", dpi=150)
plt.show()

# Fase 4: Clasificación

## 8. Estrategia de Validación Cruzada

In [ ]:
print("\n" + "=" * 60)
print("Estrategia de Validación Cruzada")
print("=" * 60)

print(f"""
Por qué la CV a nivel de sujeto es crítica:
--------------------------------------------
1. Múltiples grabaciones por sujeto comparten patrones específicos del sujeto
2. Si el mismo sujeto aparece en entrenamiento y prueba, el modelo aprende identidad del sujeto
3. Esto causa fuga de datos y precisión inflada
4. La CV a nivel de sujeto asegura generalización a NUEVOS sujetos

Estrategia: GroupKFold (n_particiones={N_PARTICIONES})
- Grupos = sujetos
- Cada fold: ~{100 // N_PARTICIONES}% de sujetos en conjunto de prueba
- Ningún sujeto aparece en entrenamiento y prueba simultáneamente
- Más robusto que Leave-One-Subject-Out (conjuntos de prueba más grandes)
""")

# Configurar CV
gkf = GroupKFold(n_splits=N_PARTICIONES)

# Verificar particiones de CV
print("Verificando particiones de CV...")
for fold, (idx_entrenamiento, idx_prueba) in enumerate(gkf.split(X_escalado, y, groups=sujetos)):
    sujetos_entrenamiento = set(sujetos[idx_entrenamiento])
    sujetos_prueba = set(sujetos[idx_prueba])
    solapamiento = sujetos_entrenamiento.intersection(sujetos_prueba)
    
    print(f"  Fold {fold + 1}:")
    print(f"    Entrenamiento: {len(idx_entrenamiento)} muestras, {len(sujetos_entrenamiento)} sujetos")
    print(f"    Prueba: {len(idx_prueba)} muestras, {len(sujetos_prueba)} sujetos")
    print(f"    Solapamiento de sujetos: {len(solapamiento)} (debería ser 0)")

## 9. Entrenamiento y Evaluación del Modelo

In [ ]:
print("\n" + "=" * 60)
print("Entrenamiento y Evaluación del Modelo")
print("=" * 60)

# Inicializar modelo
modelo_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=SEMILLA_ALEATORIA,
    n_jobs=-1
)

# Puntajes de validación cruzada
print(f"\nEntrenando Random Forest con GroupKFold CV (k={N_PARTICIONES})...")
puntajes_cv = cross_val_score(
    modelo_rf, X_escalado, y, groups=sujetos, cv=gkf, scoring="accuracy"
)

print(f"\nResultados de Validación Cruzada:")
print(f"  Precisión por fold: {puntajes_cv}")
print(f"  Precisión media: {puntajes_cv.mean():.3f} +/- {puntajes_cv.std():.3f}")
print(f"  Mín/Máx: {puntajes_cv.min():.3f} / {puntajes_cv.max():.3f}")

# Obtener predicciones de CV para matriz de confusión (¡IMPORTANTE: no predicciones de entrenamiento!)
print("\nObteniendo predicciones validadas cruzadamente...")
y_pred_cv = cross_val_predict(modelo_rf, X_escalado, y, groups=sujetos, cv=gkf)

# Calcular métricas adicionales
f1_cv = f1_score(y, y_pred_cv, average="weighted")
print(f"  Puntaje F1 (ponderado): {f1_cv:.3f}")

# Intentar calcular AUC si es posible
try:
    # Obtener predicciones de probabilidad
    y_proba_cv = cross_val_predict(
        modelo_rf, X_escalado, y, groups=sujetos, cv=gkf, method="predict_proba"
    )
    auc_cv = roc_auc_score(y, y_proba_cv[:, 1])
    print(f"  ROC AUC: {auc_cv:.3f}")
except Exception as e:
    print(f"  ROC AUC: No se pudo calcular ({e})")
    auc_cv = None

# Reporte de clasificación (en predicciones de CV)
print("\nReporte de Clasificación (Validación Cruzada):")
print(classification_report(y, y_pred_cv, target_names=["Lento", "Rápido"]))

## 10. Matriz de Confusión (Validación Cruzada)

In [ ]:
# ¡IMPORTANTE: Usando predicciones de CV, no predicciones de entrenamiento!
mc = confusion_matrix(y, y_pred_cv)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    mc,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["Lento", "Rápido"],
    yticklabels=["Lento", "Rápido"],
    ax=ax,
    annot_kws={"size": 16}
)
ax.set_xlabel("Predicción", fontsize=12, fontweight="bold")
ax.set_ylabel("Real", fontsize=12, fontweight="bold")
ax.set_title("Matriz de Confusión (Predicciones de Validación Cruzada)", fontsize=14, fontweight="bold")

# Añadir texto con métricas
texto_metricas = f"Precisión CV: {puntajes_cv.mean():.1%}\nPuntaje F1: {f1_cv:.3f}"
props = dict(boxstyle="round", facecolor="wheat", alpha=0.8)
ax.text(1.35, 0.5, texto_metricas, transform=ax.transAxes, fontsize=12,
        verticalalignment="center", bbox=props)

plt.tight_layout()
plt.savefig(DIRECTORIO_RESULTADOS / "matriz_confusion_cv.png", dpi=150)
plt.show()

print("Nota: Esta matriz de confusión usa predicciones de VALIDACIÓN CRUZADA,")
print("no predicciones del conjunto de entrenamiento. Esto da una estimación honesta del rendimiento.")

## 11. Análisis de Importancia de Características

In [ ]:
print("\n" + "=" * 60)
print("Análisis de Importancia de Características")
print("=" * 60)

# Entrenar en conjunto completo para importancia de características
modelo_rf.fit(X_escalado, y)
importancia_caracteristicas = modelo_rf.feature_importances_

# Crear DataFrame de importancia de características
df_importancia = pd.DataFrame({
    "caracteristica": nombres_caracteristicas,
    "importancia": importancia_caracteristicas
}).sort_values("importancia", ascending=False)

# Extraer información de banda y dimensión
df_importancia["banda"] = df_importancia["caracteristica"].apply(
    lambda x: x.split("_")[0] if "_" in x else "desconocido"
)
df_importancia["dimension"] = df_importancia["caracteristica"].apply(
    lambda x: "H0" if "_h0_" in x else "H1" if "_h1_" in x else "desconocido"
)

# Top 15 características
print("\nTop 15 Características Más Importantes:")
print("-" * 60)
for i, fila in df_importancia.head(15).iterrows():
    print(f"  {fila['caracteristica']}: {fila['importancia']:.4f}")

# Visualizar características principales
fig, ejes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Top 15 características
ax1 = ejes[0]
top_15 = df_importancia.head(15)
colores = ["blue" if "h0" in c else "orange" for c in top_15["caracteristica"]]
ax1.barh(range(15), top_15["importancia"].values, color=colores, alpha=0.7)
ax1.set_yticks(range(15))
ax1.set_yticklabels(top_15["caracteristica"].values, fontsize=9)
ax1.set_xlabel("Importancia", fontsize=12)
ax1.set_title("Top 15 Características Más Importantes", fontsize=14, fontweight="bold")
ax1.invert_yaxis()

# Añadir leyenda
from matplotlib.patches import Patch
elementos_leyenda = [Patch(facecolor="blue", alpha=0.7, label="H0 (componentes)"),
                     Patch(facecolor="orange", alpha=0.7, label="H1 (ciclos)")]
ax1.legend(handles=elementos_leyenda, loc="lower right")

# Gráfico 2: Importancia por banda de frecuencia
ax2 = ejes[1]
importancia_banda = df_importancia.groupby("banda")["importancia"].sum().sort_values(ascending=True)
colores_banda = plt.cm.viridis(np.linspace(0.2, 0.8, len(importancia_banda)))
ax2.barh(importancia_banda.index, importancia_banda.values, color=colores_banda)
ax2.set_xlabel("Importancia Total", fontsize=12)
ax2.set_title("Importancia por Banda de Frecuencia", fontsize=14, fontweight="bold")

# Añadir etiquetas de porcentaje
importancia_total = importancia_banda.sum()
for i, (banda, imp) in enumerate(importancia_banda.items()):
    ax2.text(imp + 0.01, i, f"{imp / importancia_total * 100:.1f}%", va="center", fontsize=10)

plt.tight_layout()
plt.savefig(DIRECTORIO_RESULTADOS / "importancia_caracteristicas.png", dpi=150)
plt.show()

# Resumen por banda y dimensión
print("\nResumen de Importancia por Banda:")
resumen_banda = df_importancia.groupby("banda")["importancia"].agg(["sum", "mean"]).sort_values("sum", ascending=False)
resumen_banda["porcentaje"] = resumen_banda["sum"] / resumen_banda["sum"].sum() * 100
print(resumen_banda.round(4))

print("\nResumen de Importancia por Dimensión:")
resumen_dim = df_importancia.groupby("dimension")["importancia"].agg(["sum", "mean"])
resumen_dim["porcentaje"] = resumen_dim["sum"] / resumen_dim["sum"].sum() * 100
print(resumen_dim.round(4))

# Fase 5: Validación Estadística

## 12. Prueba de Permutación

In [ ]:
print("\n" + "=" * 60)
print("Validación Estadística: Prueba de Permutación")
print("=" * 60)

print("""
Prueba de Permutación:
----------------------
H0 (hipótesis nula): La precisión de clasificación se debe al azar
H1 (alternativa): Las características topológicas contienen información discriminativa

Método: Permutar etiquetas N veces, calcular precisión CV cada vez
Valor p: Proporción de precisiones permutadas >= precisión observada
""")

def prueba_permutacion_cv(X, y, grupos, modelo, cv, n_permutaciones=1000, semilla=42):
    """Prueba de permutación para precisión de validación cruzada."""
    # Puntaje observado
    puntajes_observados = cross_val_score(modelo, X, y, groups=grupos, cv=cv, scoring="accuracy")
    media_observada = puntajes_observados.mean()
    
    # Distribución nula
    np.random.seed(semilla)
    distribucion_nula = []
    
    for i in tqdm(range(n_permutaciones), desc="Prueba de permutación"):
        y_permutado = np.random.permutation(y)
        puntajes_perm = cross_val_score(modelo, X, y_permutado, groups=grupos, cv=cv, scoring="accuracy")
        distribucion_nula.append(puntajes_perm.mean())
    
    distribucion_nula = np.array(distribucion_nula)
    
    # Valor p (una cola, probando si observado > nulo)
    valor_p = (np.sum(distribucion_nula >= media_observada) + 1) / (n_permutaciones + 1)
    
    # Tamaño del efecto (d de Cohen)
    tamano_efecto = (media_observada - distribucion_nula.mean()) / distribucion_nula.std()
    
    return media_observada, distribucion_nula, valor_p, tamano_efecto


# Ejecutar prueba de permutación
precision_observada, dist_nula, valor_p, tamano_efecto = prueba_permutacion_cv(
    X_escalado, y, sujetos, modelo_rf, gkf, 
    n_permutaciones=N_PERMUTACIONES, 
    semilla=SEMILLA_ALEATORIA
)

print(f"\nResultados de Prueba de Permutación:")
print(f"  Precisión CV observada: {precision_observada:.4f} ({precision_observada:.1%})")
print(f"  Media distribución nula: {dist_nula.mean():.4f} ({dist_nula.mean():.1%})")
print(f"  Std distribución nula: {dist_nula.std():.4f}")
print(f"  Valor p: {valor_p:.6f}")
print(f"  Tamaño del efecto (d de Cohen): {tamano_efecto:.2f}")

# Interpretar tamaño del efecto
if abs(tamano_efecto) < 0.2:
    interpretacion_efecto = "insignificante"
elif abs(tamano_efecto) < 0.5:
    interpretacion_efecto = "pequeño"
elif abs(tamano_efecto) < 0.8:
    interpretacion_efecto = "mediano"
else:
    interpretacion_efecto = "grande"
print(f"  Interpretación del efecto: {interpretacion_efecto}")

# Interpretar significancia
alfa = 0.05
if valor_p < 0.001:
    nivel_sig = "*** (p < 0.001)"
elif valor_p < 0.01:
    nivel_sig = "** (p < 0.01)"
elif valor_p < 0.05:
    nivel_sig = "* (p < 0.05)"
else:
    nivel_sig = "ns (p >= 0.05)"
print(f"  Significancia: {nivel_sig}")

## 13. Intervalo de Confianza Bootstrap

In [ ]:
print("\n" + "=" * 60)
print("Validación Estadística: Intervalo de Confianza Bootstrap")
print("=" * 60)

def bootstrap_puntaje_cv(X, y, grupos, modelo, cv, n_bootstrap=1000, semilla=42):
    """Intervalo de confianza bootstrap para precisión CV mediante remuestreo de sujetos."""
    np.random.seed(semilla)
    puntajes_bootstrap = []
    
    sujetos_unicos = np.unique(grupos)
    n_sujetos = len(sujetos_unicos)
    
    for i in tqdm(range(n_bootstrap), desc="Bootstrap"):
        # Remuestrear sujetos con reemplazo
        sujetos_boot = np.random.choice(sujetos_unicos, size=n_sujetos, replace=True)
        
        # Obtener índices para sujetos seleccionados
        indices_boot = []
        nuevos_grupos = []
        for j, sujeto in enumerate(sujetos_boot):
            indices_sujeto = np.where(grupos == sujeto)[0]
            indices_boot.extend(indices_sujeto)
            # Asignar nuevo ID de grupo para manejar sujetos duplicados
            nuevos_grupos.extend([j] * len(indices_sujeto))
        
        indices_boot = np.array(indices_boot)
        nuevos_grupos = np.array(nuevos_grupos)
        
        X_boot = X[indices_boot]
        y_boot = y[indices_boot]
        
        # Verificar que tenemos ambas clases y suficientes grupos
        if len(np.unique(y_boot)) < 2:
            continue
        if len(np.unique(nuevos_grupos)) < cv.n_splits:
            continue
        
        try:
            puntajes_boot = cross_val_score(
                modelo, X_boot, y_boot, groups=nuevos_grupos, cv=cv, scoring="accuracy"
            )
            puntajes_bootstrap.append(puntajes_boot.mean())
        except Exception:
            continue
    
    return np.array(puntajes_bootstrap)


# Ejecutar bootstrap
precisiones_bootstrap = bootstrap_puntaje_cv(
    X_escalado, y, sujetos, modelo_rf, gkf,
    n_bootstrap=N_BOOTSTRAP,
    semilla=SEMILLA_ALEATORIA
)

# Intervalo de confianza (método de percentiles)
ic_inferior = np.percentile(precisiones_bootstrap, 2.5)
ic_superior = np.percentile(precisiones_bootstrap, 97.5)
ancho_ic = ic_superior - ic_inferior

print(f"\nResultados Bootstrap ({len(precisiones_bootstrap)} iteraciones exitosas):")
print(f"  Precisión media: {precisiones_bootstrap.mean():.4f} ({precisiones_bootstrap.mean():.1%})")
print(f"  Std: {precisiones_bootstrap.std():.4f}")
print(f"  Mediana: {np.median(precisiones_bootstrap):.4f}")
print(f"\nIntervalo de Confianza 95%:")
print(f"  [{ic_inferior:.4f}, {ic_superior:.4f}]")
print(f"  [{ic_inferior:.1%}, {ic_superior:.1%}]")
print(f"  Ancho: {ancho_ic:.4f} ({ancho_ic:.1%})")

# Verificar si IC excluye el azar
if ic_inferior > 0.5:
    print(f"\n  ✓ El IC completo está sobre el azar (50%) - resultado robusto")
else:
    print(f"\n  ⚠ El IC incluye el nivel de azar (50%) - el resultado puede no ser robusto")

## 14. Visualización de Pruebas Estadísticas

In [ ]:
fig, ejes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Prueba de Permutación
ax1 = ejes[0]
ax1.hist(dist_nula, bins=50, alpha=0.7, color="gray", edgecolor="black", density=True,
         label=f"Distribución nula (n={N_PERMUTACIONES})")
ax1.axvline(precision_observada, color="red", linewidth=3, linestyle="--",
            label=f"Observada ({precision_observada:.1%})")
ax1.axvline(dist_nula.mean(), color="blue", linewidth=2, linestyle=":",
            label=f"Media nula ({dist_nula.mean():.1%})")
ax1.axvline(0.5, color="green", linewidth=2, linestyle="-.",
            label="Azar (50%)")

ax1.set_xlabel("Precisión de Validación Cruzada", fontsize=12, fontweight="bold")
ax1.set_ylabel("Densidad", fontsize=12, fontweight="bold")
ax1.set_title("Prueba de Permutación: Distribución Nula", fontsize=14, fontweight="bold")
ax1.legend(loc="upper left", fontsize=10)
ax1.grid(True, alpha=0.3)

# Añadir anotación de valor p
texto_p = f"p = {valor_p:.4f}\nd de Cohen = {tamano_efecto:.2f}\n{nivel_sig}"
props = dict(boxstyle="round", facecolor="wheat", alpha=0.9)
ax1.text(0.97, 0.97, texto_p, transform=ax1.transAxes, fontsize=11,
         verticalalignment="top", horizontalalignment="right", bbox=props)

# Gráfico 2: Distribución Bootstrap
ax2 = ejes[1]
ax2.hist(precisiones_bootstrap, bins=50, alpha=0.7, color="steelblue", edgecolor="black", density=True,
         label=f"Distribución bootstrap (n={len(precisiones_bootstrap)})")
ax2.axvline(precision_observada, color="red", linewidth=3, linestyle="--",
            label=f"Observada ({precision_observada:.1%})")
ax2.axvline(ic_inferior, color="orange", linewidth=2, linestyle=":",
            label=f"IC 95%: [{ic_inferior:.1%}, {ic_superior:.1%}]")
ax2.axvline(ic_superior, color="orange", linewidth=2, linestyle=":")
ax2.axvspan(ic_inferior, ic_superior, alpha=0.2, color="orange")
ax2.axvline(0.5, color="green", linewidth=2, linestyle="-.", label="Azar (50%)")

ax2.set_xlabel("Precisión de Validación Cruzada", fontsize=12, fontweight="bold")
ax2.set_ylabel("Densidad", fontsize=12, fontweight="bold")
ax2.set_title("Bootstrap: Intervalo de Confianza 95%", fontsize=14, fontweight="bold")
ax2.legend(loc="upper left", fontsize=10)
ax2.grid(True, alpha=0.3)

# Añadir anotación de IC
texto_ic = f"IC 95%:\n[{ic_inferior:.1%}, {ic_superior:.1%}]\nAncho: {ancho_ic:.1%}"
props = dict(boxstyle="round", facecolor="lightblue", alpha=0.9)
ax2.text(0.97, 0.97, texto_ic, transform=ax2.transAxes, fontsize=11,
         verticalalignment="top", horizontalalignment="right", bbox=props)

plt.tight_layout()
plt.savefig(DIRECTORIO_RESULTADOS / "pruebas_estadisticas.png", dpi=150)
plt.show()

## 15. Resultados Finales y Conclusiones

In [ ]:
print("\n" + "=" * 70)
print("RESULTADOS FINALES Y CONCLUSIONES")
print("=" * 70)

# Tabla resumen
print("\n" + "-" * 70)
print("TABLA RESUMEN")
print("-" * 70)

resumen_resultados = {
    "Métrica": [
        "Tamaño del conjunto de datos",
        "Número de características",
        "Número de sujetos",
        "Balance de clases (Lento/Rápido)",
        "",
        "Precisión CV (GroupKFold-5)",
        "Puntaje F1 (ponderado)",
        "Línea base (azar)",
        "Mejora sobre línea base",
        "",
        "Valor p (prueba de permutación)",
        "Tamaño del efecto (d de Cohen)",
        "Límite inferior IC 95%",
        "Límite superior IC 95%",
        "¿IC sobre el azar?",
    ],
    "Valor": [
        f"{X.shape[0]} muestras",
        f"{X.shape[1]} características",
        f"{len(np.unique(sujetos))} sujetos",
        f"{np.sum(y == 0)} / {np.sum(y == 1)}",
        "",
        f"{puntajes_cv.mean():.1%} ± {puntajes_cv.std():.1%}",
        f"{f1_cv:.3f}",
        "50%",
        f"+{(puntajes_cv.mean() - 0.5) * 100:.1f} puntos porcentuales",
        "",
        f"{valor_p:.6f} {nivel_sig}",
        f"{tamano_efecto:.2f} ({interpretacion_efecto})",
        f"{ic_inferior:.1%}",
        f"{ic_superior:.1%}",
        "Sí" if ic_inferior > 0.5 else "No",
    ],
}

df_resultados = pd.DataFrame(resumen_resultados)
print(df_resultados.to_string(index=False))

# Bandas más importantes
print("\n" + "-" * 70)
print("BANDAS DE FRECUENCIA MÁS DISCRIMINATIVAS")
print("-" * 70)
print(resumen_banda.round(3).to_string())

# Interpretación
print("\n" + "-" * 70)
print("INTERPRETACIÓN")
print("-" * 70)

print(f"""
Pregunta de Investigación:
  ¿Podemos distinguir entre condiciones de audio lento y rápido en infantes
  basándonos en la topología de conectividad EEG?

Resultados:
  - Precisión validada cruzadamente: {puntajes_cv.mean():.1%} (azar = 50%)
  - Esto es {(puntajes_cv.mean() - 0.5) * 100:.1f} puntos porcentuales sobre el azar
  - Valor p = {valor_p:.6f} → {"estadísticamente significativo" if valor_p < 0.05 else "no estadísticamente significativo"}
  - Tamaño del efecto = {tamano_efecto:.2f} → significancia práctica {interpretacion_efecto}
  - IC 95% = [{ic_inferior:.1%}, {ic_superior:.1%}]
""")

# Nivel de evidencia
if puntajes_cv.mean() > 0.65 and valor_p < 0.05 and ic_inferior > 0.5:
    evidencia = "FUERTE"
    conclusion = """
Las características topológicas de los grafos de conectividad EEG distinguen exitosamente
entre condiciones de audio lento y rápido. Esto sugiere que:

1. El audio lento vs rápido induce patrones de conectividad cerebral mediblemente diferentes
2. Estas diferencias son robustas entre sujetos (no artefactos específicos del sujeto)
3. El Análisis de Datos Topológicos captura diferencias significativas en la señal neural

Las bandas de frecuencia más discriminativas proporcionan información sobre qué tipos de
oscilaciones neurales son más afectadas por la velocidad del audio."""

elif puntajes_cv.mean() > 0.55 and valor_p < 0.05:
    evidencia = "MODERADA"
    conclusion = """
Hay evidencia moderada de que las características topológicas pueden distinguir condiciones.
La precisión está sobre el azar y es estadísticamente significativa, pero el tamaño
del efecto es modesto. Esto sugiere:

1. Existen algunas diferencias en topología de conectividad entre condiciones
2. La señal puede ser débil o variable entre sujetos
3. Considerar: más sujetos, diferentes características, o enfoques alternativos"""

else:
    evidencia = "DÉBIL/NINGUNA"
    conclusion = """
Las características topológicas no distinguen confiablemente entre condiciones.
Posibles explicaciones:

1. La velocidad del audio puede no afectar significativamente la topología de conectividad EEG
2. El efecto existe pero es demasiado sutil para los métodos actuales
3. Se necesitan más sujetos o diferente preprocesamiento
4. Se podrían explorar enfoques TDA alternativos (ej., diferentes filtraciones)"""

print(f"Nivel de Evidencia: {evidencia}")
print(conclusion)

# Guardar resultados
diccionario_resultados = {
    "precision_cv_media": puntajes_cv.mean(),
    "precision_cv_std": puntajes_cv.std(),
    "puntaje_f1_cv": f1_cv,
    "valor_p": valor_p,
    "tamano_efecto": tamano_efecto,
    "ic_inferior": ic_inferior,
    "ic_superior": ic_superior,
    "n_muestras": X.shape[0],
    "n_caracteristicas": X.shape[1],
    "n_sujetos": len(np.unique(sujetos)),
    "nivel_evidencia": evidencia,
}

# Guardar como JSON
import json
with open(DIRECTORIO_RESULTADOS / "resumen_resultados.json", "w") as f:
    json.dump(diccionario_resultados, f, indent=2)

print(f"\nResultados guardados en {DIRECTORIO_RESULTADOS}/")
print("  - matriz_confusion_cv.png")
print("  - importancia_caracteristicas.png")
print("  - pruebas_estadisticas.png")
print("  - distribucion_sujetos.png")
print("  - diagrama_persistencia_ejemplo.png")
print("  - resumen_resultados.json")

print("\n" + "=" * 70)
print("ANÁLISIS COMPLETADO")
print("=" * 70)